In [1]:
# %pip install graphviz
# %pip install pydotplus
# %pip install heamy
# %pip install mlxtend
# %pip install git+git://github.com/kvoyager/GmdhPy.git

In [42]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,cross_val_score, LeaveOneOut, GridSearchCV, learning_curve, validation_curve
from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score ,recall_score, precision_score
from IPython.core.display import HTML
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import export_text
%matplotlib inline 
sns.set(style="ticks")
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC, NuSVC, LinearSVC, OneClassSVM, SVR, NuSVR, LinearSVR
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from IPython.core.display import HTML
from sklearn.tree import export_text
from IPython.display import Image
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor, BaggingRegressor

from IPython.display import Image
import graphviz
import pydotplus
from io import StringIO
from heamy.estimator import Regressor, Classifier
from heamy.pipeline import ModelsPipeline
from heamy.dataset import Dataset
from mlxtend.classifier import StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
# from gmdhpy import gmdh


In [3]:
dataset = pd.read_csv('./diamonds.csv')
dataset.head()

,carat,cut,color,clarity,depth,table,price,'x','y','z'
0,0.23,b'Ideal',b'E',b'SI2',61.5,55.0,326.0,3.95,3.98,2.43
1,0.21,b'Premium',b'E',b'SI1',59.8,61.0,326.0,3.89,3.84,2.31
2,0.23,b'Good',b'E',b'VS1',56.9,65.0,327.0,4.05,4.07,2.31
3,0.29,b'Premium',b'I',b'VS2',62.4,58.0,334.0,4.20,4.23,2.63
4,0.31,b'Good',b'J',b'SI2',63.3,58.0,335.0,4.34,4.35,2.75


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  float64
 7   'x'      53940 non-null  float64
 8   'y'      53940 non-null  float64
 9   'z'      53940 non-null  float64
dtypes: float64(7), object(3)
memory usage: 4.1+ MB


In [5]:
dataset.cut.replace("b'Fair'",1,inplace=True)
dataset.cut.replace("b'Good'",2,inplace=True)
dataset.cut.replace("b'Very Good'",3,inplace=True)
dataset.cut.replace("b'Ideal'",4,inplace=True)
dataset.cut.replace("b'Premium'",5,inplace=True)
dataset.clarity.replace("b'I1'",1,inplace=True)
dataset.clarity.replace("b'IF'",2,inplace=True)
dataset.clarity.replace("b'SI1'",3,inplace=True)
dataset.clarity.replace("b'SI2'",4,inplace=True)
dataset.clarity.replace("b'VS1'",5,inplace=True)
dataset.clarity.replace("b'VS2'",6,inplace=True)
dataset.clarity.replace("b'VVS1'",7,inplace=True)
dataset.clarity.replace("b'VVS2'",8,inplace=True)
dataset.color.replace("b'J'",1,inplace=True)
dataset.color.replace("b'I'",2,inplace=True)
dataset.color.replace("b'H'",3,inplace=True)
dataset.color.replace("b'G'",4,inplace=True)
dataset.color.replace("b'F'",5,inplace=True)
dataset.color.replace("b'E'",6,inplace=True)
dataset.color.replace("b'D'",7,inplace=True)

In [7]:
shuffled = dataset.sample(frac=1)
shuffled = shuffled[:5000]
shuffled.shape
data_x = shuffled.iloc[:, [0,1,2,3,4,5,7,8,9]].values
data_y = shuffled.iloc[:, 6].values
data_x

array([[0.33, 3.  , 3.  , ..., 4.44, 4.47, 2.68],
       [0.4 , 5.  , 7.  , ..., 4.69, 4.74, 2.91],
       [1.01, 3.  , 7.  , ..., 6.35, 6.46, 4.03],
       ...,
       [1.51, 3.  , 6.  , ..., 7.26, 7.3 , 4.57],
       [1.63, 5.  , 4.  , ..., 7.64, 7.58, 4.64],
       [0.56, 4.  , 7.  , ..., 5.3 , 5.26, 3.27]])

In [8]:
data_y

array([  579.,   666.,  4760., ..., 11460., 11721.,  2066.])

In [9]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size = 0.2,shuffle=True)

In [49]:
bc1 = BaggingRegressor(n_estimators=5, oob_score=True, random_state=10)
bc1.fit(data_x, data_y)

D:\Projects\python\ml\venv\lib\site-packages\sklearn\ensemble\_bagging.py:1253: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(


BaggingRegressor(n_estimators=5, oob_score=True, random_state=10)

In [50]:
tree_rules = export_text(bc1[0], feature_names=list(dataset.iloc[:, [0,1,2,3,4,5,7,8,9]].columns))
HTML('<pre>' + tree_rules + '</pre>')

In [51]:
bc1.estimators_samples_

[array([1445, 3508, 3943, ..., 3299, 4109, 1256]),
 array([ 814, 1770,  861, ..., 4166, 2322,  471]),
 array([1433, 2043,  643, ..., 1200, 2847, 2132]),
 array([2646, 3895,  271, ..., 4601, 4609, 4314]),
 array([4752, 1806, 3518, ..., 4714, 4568, 4333])]

In [52]:
# Сконвертируем эти данные в двоичную матрицу, 
# 1 соответствует элементам, попавшим в обучающую выборку
bin_array = np.zeros((5, data_x.shape[0]))
for i in range(5):
    for j in bc1.estimators_samples_[i]:
        bin_array[i][j] = 1
bin_array

array([[1., 0., 0., ..., 1., 1., 1.],
       [0., 1., 0., ..., 1., 1., 1.],
       [1., 1., 1., ..., 0., 1., 0.],
       [0., 0., 1., ..., 1., 1., 1.],
       [0., 1., 1., ..., 1., 1., 1.]])

In [53]:
# Оценим Out-of-bag error, теоретическое значение 37%
for i in range(5):
    cur_data = bin_array[i]
    len_cur_data = len(cur_data)
    sum_cur_data = sum(cur_data)
    (len(bin_array[0]) - sum(bin_array[0])) / len(bin_array[0])
    oob_i = (len_cur_data - sum_cur_data) / len_cur_data
    print('Для модели № {} размер OOB составляет {}%'.format(i+1, round(oob_i, 4)*100.0))

Для модели № 1 размер OOB составляет 35.839999999999996%
Для модели № 2 размер OOB составляет 36.720000000000006%
Для модели № 3 размер OOB составляет 37.08%
Для модели № 4 размер OOB составляет 36.5%
Для модели № 5 размер OOB составляет 36.58%


In [54]:
# Out-of-bag error, возвращаемый классификатором
# Для классификации используется метрика accuracy
bc1.oob_score_, 1-bc1.oob_score_

(0.7599016530299646, 0.2400983469700354)

In [56]:
tree1 = RandomForestRegressor(n_estimators=5, oob_score=True, random_state=10)
tree1.fit(data_x, data_y)

D:\Projects\python\ml\venv\lib\site-packages\sklearn\ensemble\_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


RandomForestRegressor(n_estimators=5, oob_score=True, random_state=10)

In [57]:
tree_rules2 = export_text(tree1[0], feature_names=list(dataset.iloc[:, [0,1,2,3,4,5,7,8,9]].columns))
HTML('<pre>' + tree_rules2 + '</pre>')

In [58]:
# Out-of-bag error, возвращаемый классификатором
tree1.oob_score_, 1-tree1.oob_score_

(0.7614338965076692, 0.23856610349233076)

In [59]:
BagModel = BaggingRegressor(n_estimators=5, oob_score=True, random_state=1)

In [60]:
BagModel.fit(x_train, y_train)

D:\Projects\python\ml\venv\lib\site-packages\sklearn\ensemble\_bagging.py:1253: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(


BaggingRegressor(n_estimators=5, oob_score=True, random_state=1)

In [61]:
y_bag_predict = BagModel.predict(x_test)
y_bag_predict

array([ 3015.8,  1967. ,  3338.2,  5255.6,  2338.2,  8480. ,   667.4,
        5754.2,  4000.6,  2754.4,  3899.4,   754.8,  1844. ,  4492.2,
        1073. ,  8299. ,  2179.6,  2662. ,  3089.6,   978.8,  1658.8,
         438.2,   915.2,  2332.2,  1552.6,   429.8,  1035.4,   845.8,
         803.6,   598.4,  2627. ,  7463. , 15025.4,  1062.6,  4839.6,
        1588.4,  5059.8,  4914. ,  9406.2,  1150.6,   950.2,  8128.2,
        5317.8,  1172.2,   662.4,  3620.2,  1030. ,  3282. ,  1047.8,
         894.4,  6331.4,   584.6,   943.4,  1269.6,   908. ,   877. ,
        3710.4,   894. ,  6843. ,  1727. ,  2289.8,  2712. ,   623.8,
        2977.4,  1274.2,  1565.8,  3497.4,  2091.4,  4996.2,   840.2,
        5016. ,  2233.6,  3884.8,   525.8,   572.4,  9486.6,  6624.2,
        4952. ,  3214.8,  1119.2,  1784.6,   928. ,   808.2,   650.2,
        2227.8,  7090. ,  3897. ,  2731.4,  1054.4,  4218.4,   599.4,
         518.6,  4034.8,  2986.6,   799.4,  3148. ,  9545. ,  1826.6,
         855.6,  231

In [62]:
r2_score(y_test, y_bag_predict)

0.9612801448566596

In [64]:
BagModel.oob_score_

0.7846669298929987

In [35]:
r2_score(y_test, y_boost_predict)

-0.6710928470279125

In [ ]:
gb1 = GradientBoostingRegressor(random_state=1)
gb1.fit(x_train, y_train)